### 01. 데이터 전처리

#### 01-1. 서울시 스타벅스 매장 목록, 인구, 사업체 데이터에 시군구명, 시군구코드 추가

In [2]:
import pandas as pd

In [3]:
# 서울시 스타벅스 매장 목록이 담긴 파일 불러오기
seoul_starbucks = pd.read_excel('file/seoul_starbucks_df.xlsx', header = 0)
seoul_starbucks.head()

,매장명,위도,경도,매장 타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,pin_general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,pin_general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.513931,127.020606,pin_general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,pin_general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩,37.494668,127.062583,pin_general,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [4]:
# 스타벅스 주소 정보에서 시군구명 출력
sgg_names = []
for address in seoul_starbucks['주소']:
    sgg = address.split()[1]
    sgg_names.append(sgg)

seoul_starbucks['시군구명'] = sgg_names
seoul_starbucks.head()

,매장명,위도,경도,매장 타입,주소,전화번호,시군구명
0,역삼아레나빌딩,37.501087,127.043069,pin_general,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
1,논현역사거리,37.510178,127.022223,pin_general,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
2,신사역성일빌딩,37.513931,127.020606,pin_general,서울특별시 강남구 강남대로 584 (논현동),1522-3232,강남구
3,국기원사거리,37.499517,127.031495,pin_general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,강남구
4,대치재경빌딩,37.494668,127.062583,pin_general,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,강남구


In [5]:
seoul_starbucks.to_excel('./file/seoul_starbucks_list.xlsx', index = False)

#### 01-2. 스타벅스 분석 데이터 만들기

In [17]:
# 시군구 목록 데이터 불러오기
seoul_sgg = pd.read_excel('./file/seoul_sgg_list.xlsx')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.665861,127.031767
1,11380,은평구,37.617612,126.922700
2,11230,동대문구,37.583801,127.050700
3,11590,동작구,37.496504,126.944307
4,11545,금천구,37.460097,126.900155


In [18]:
df = seoul_starbucks[['시군구명', '매장명']].groupby('시군구명').count()

In [19]:
pd.merge(seoul_sgg, df, on = '시군구명', how = 'left').rename(columns={'매장명':'매장수'}).head()

,시군구코드,시군구명,위도,경도,매장수
0,11320,도봉구,37.665861,127.031767,7
1,11380,은평구,37.617612,126.922700,14
2,11230,동대문구,37.583801,127.050700,13
3,11590,동작구,37.496504,126.944307,11
4,11545,금천구,37.460097,126.900155,13


In [20]:
#------------------- 수업
starbucks_sgg_count = seoul_starbucks.pivot_table(index = '시군구명',
                                                  values = '매장명', 
                                                  aggfunc = 'count').rename(columns = {'매장명':'스타벅스_매장수'})
starbucks_sgg_count.head()

,스타벅스_매장수
시군구명,
강남구,91
강동구,17
강북구,6
강서구,27
관악구,12


In [21]:
# 서울시 시군구 목록 데이터에 스타벅스 매장 수 데이터를 병합
seoul_sgg = pd.merge(seoul_sgg, starbucks_sgg_count, how = 'left', on = '시군구명')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장수
0,11320,도봉구,37.665861,127.031767,7
1,11380,은평구,37.617612,126.922700,14
2,11230,동대문구,37.583801,127.050700,13
3,11590,동작구,37.496504,126.944307,11
4,11545,금천구,37.460097,126.900155,13


In [ ]:
seoul_sgg_pop = pd.read_excel('./file/sgg_pop_df_final.xlsx')
seoul_sgg_pop

In [23]:
seoul_sgg = pd.merge(seoul_sgg, seoul_sgg_pop, how = 'left', on = '시군구명')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민등록인구
0,11320,도봉구,37.665861,127.031767,7,312858
1,11380,은평구,37.617612,126.922700,14,468766
2,11230,동대문구,37.583801,127.050700,13,354884
3,11590,동작구,37.496504,126.944307,11,390377
4,11545,금천구,37.460097,126.900155,13,242467


In [26]:
# 서울시 시군구 목록 데이터에 서울시 시군구별 사업체 수 통계 데이터를 병합
seoul_sgg_biz = pd.read_excel('file/sgg_biz_df_final.xlsx')
seoul_sgg = pd.merge(seoul_sgg,seoul_sgg_biz, how='left', on = '시군구명')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민등록인구,종사자수,사업체수
0,11320,도봉구,37.665861,127.031767,7,312858,79097,25391
1,11380,은평구,37.617612,126.922700,14,468766,109031,36509
2,11230,동대문구,37.583801,127.050700,13,354884,146383,42813
3,11590,동작구,37.496504,126.944307,11,390377,109281,27915
4,11545,금천구,37.460097,126.900155,13,242467,255449,47964


In [27]:
seoul_sgg.to_excel('./file/seoul_sgg_stat.xlsx', index = False)